# Convolution demos

In this notebook, we use the batch, multi-channel convolution operation implemented in Numpy (that you can find [here](../lincoln/lincoln/conv.py)) to train a small convolutional neural network to more than 90% accuracy on MNIST.

In [2]:
import numpy as np

import lincoln
from lincoln.layers import Dense
from lincoln.losses import SoftmaxCrossEntropy, MeanSquaredError
from lincoln.optimizers import Optimizer, SGD, SGDMomentum
from lincoln.activations import Sigmoid, Tanh, Linear, ReLU
from lincoln.network import NeuralNetwork
from lincoln.train import Trainer
from lincoln.utils import mnist
from lincoln.layers import Conv2D

X_train, y_train, X_test, y_test = mnist.load()

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
X_train, X_test = X_train - np.mean(X_train), X_test - np.mean(X_train)
X_train, X_test = X_train / np.std(X_train), X_test / np.std(X_train)

In [5]:
X_train_conv, X_test_conv = X_train.reshape(-1, 1, 28, 28), X_test.reshape(-1, 1, 28, 28)

In [6]:
num_labels = len(y_train)
train_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    train_labels[i][y_train[i]] = 1

num_labels = len(y_test)
test_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    test_labels[i][y_test[i]] = 1

In [7]:
def calc_accuracy_model(model, test_set):
    return print(f'''The model validation accuracy is: 
    {np.equal(np.argmax(model.forward(test_set, inference=True), axis=1), y_test).sum() * 100.0 / test_set.shape[0]:.2f}%''')

# CNN from scratch

In [8]:
model = NeuralNetwork(
    layers=[Conv2D(out_channels=16,
                   param_size=5,
                   dropout=0.8,
                   weight_init="glorot",
                   flatten=True,
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190402)

trainer = Trainer(model, SGDMomentum(lr = 0.1, momentum=0.9))
trainer.fit(X_train_conv, train_labels, X_test_conv, test_labels,
            epochs = 1,
            eval_every = 1,
            seed=20190402,
            batch_size=60,
            conv_testing=True);

batch 0 loss 31.191501893742398
batch 10 loss 14.150390490519676
batch 20 loss 8.507022908673962
batch 30 loss 9.816084637419738
batch 40 loss 2.7069292896566806
batch 50 loss 5.039130992880024
batch 60 loss 3.8413244521210697
batch 70 loss 8.473928496704705
batch 80 loss 5.377665360919979
batch 90 loss 2.3822865877907407
batch 100 loss 3.9719645853900385
Validation accuracy after 100 batches is 86.88%
batch 110 loss 7.9747190608183685
batch 120 loss 7.077668956662975
batch 130 loss 6.216995112179712
batch 140 loss 2.964481594125498
batch 150 loss 10.53799052041313
batch 160 loss 4.606493076829719
batch 170 loss 4.76040145549784
batch 180 loss 4.545511615017702
batch 190 loss 5.031739074892001
batch 200 loss 6.216979767439407
Validation accuracy after 200 batches is 84.74%
batch 210 loss 3.453877699981557
batch 220 loss 4.148922365607597
batch 230 loss 4.296710387843259
batch 240 loss 3.639397773886752
batch 250 loss 3.409146241965744
batch 260 loss 6.400507715600272
batch 270 loss 3.3

In [9]:
calc_accuracy_model(model, X_test_conv)

The model validation accuracy is: 
    92.66%
